In [ ]:
%load_ext autoreload
%autoreload 2
# type: ignore
from nanopyx.data.examples.download import ExampleDataManager
from nanopyx.core.transform.binning import rebin2d
from nanopyx.core.transform.image_magnify import fourier_zoom, catmull_rom_zoom
from scipy.ndimage import zoom as scipy_zoom
from skimage.transform import (
    rescale as ski_rescale, 
    resize as ski_resize, 
    downscale_local_mean as ski_downscale_local_mean
    )
from cv2 import resize as cv_resize
from cv2 import INTER_LANCZOS4

downloader = ExampleDataManager()
z = downloader.get_zarr("SMLMS2013_HDTubulinAlexa647")
image = z['data'][0]
imageHalfSize = rebin2d(image, 2, mode="mean")

In [ ]:
%%prun -s cumulative -q -l 100 -T prun_magnify.profile
# Profile the run

imageFFTZoom = fourier_zoom(imageHalfSize, 2)
imageCRZoom = catmull_rom_zoom(imageHalfSize, 2)
imageScipyZoom = scipy_zoom(imageHalfSize, 2)
imageSkiZoom = ski_rescale(imageHalfSize, 2, anti_aliasing=False)
imageCVZoom = cv_resize(imageHalfSize, None, fx=2, fy=2, interpolation = INTER_LANCZOS4)

In [ ]:
# Show profiling
print(open('prun_magnify', 'r').read())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from nanopyx.core.analysis.pearson_correlation import pearson_correlation
import numpy as np

# Create a figure with 3 subplots arranged in a row
fig, axs = plt.subplots(6, 2, figsize=(16, 3*16))
sns.heatmap(image, ax=axs[0,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(imageHalfSize, ax=axs[0,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[0,0].set_title("orginal")
axs[0,1].set_title("half-size")

error = image-imageFFTZoom
rmse = round((error**2).mean()**0.5)
pc = pearson_correlation(image, imageFFTZoom)
sns.heatmap(imageFFTZoom, ax=axs[1,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(error, ax=axs[1,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[1,0].set_title("fft-zoom")
axs[1,1].set_title("fft-zoom-error")
axs[1,1].set_xlabel(f"RMSE={rmse} PC={round(pc, 2)} MaxError={round(np.max(error))}")

error = image-imageCRZoom
rmse = round((error**2).mean()**0.5)
pc = pearson_correlation(image, imageCRZoom)
sns.heatmap(imageCRZoom, ax=axs[2,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(error, ax=axs[2,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[2,0].set_title("cr-zoom")
axs[2,1].set_title("cr-zoom-error")
axs[2,1].set_xlabel(f"RMSE={rmse} PC={round(pc, 2)} MaxError={round(np.max(error))}")

error = image-imageScipyZoom
rmse = round((error**2).mean()**0.5)
pc = pearson_correlation(image, imageScipyZoom)
sns.heatmap(imageScipyZoom, ax=axs[3,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(error, ax=axs[3,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[3,0].set_title("scipy-zoom")
axs[3,1].set_title("scipy-zoom-error")
axs[3,1].set_xlabel(f"RMSE={rmse} PC={round(pc, 2)} MaxError={round(np.max(error))}")

error = image-imageSkiZoom
rmse = round((error**2).mean()**0.5)
pc = pearson_correlation(image, imageSkiZoom)
sns.heatmap(imageSkiZoom, ax=axs[4,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(error, ax=axs[4,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[4,0].set_title("skimage-zoom")
axs[4,1].set_title("skimage-zoom-error")
axs[4,1].set_xlabel(f"RMSE={rmse} PC={round(pc, 2)} MaxError={round(np.max(error))}")


error = image-imageCVZoom
rmse = round((error**2).mean()**0.5)
pc = pearson_correlation(image, imageCVZoom)
sns.heatmap(imageCVZoom, ax=axs[5,0], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
sns.heatmap(error, ax=axs[5,1], cmap='inferno', cbar=False, square=True, xticklabels=False, yticklabels=False)
axs[5,0].set_title("cv-zoom")
axs[5,1].set_title("cv-zoom-error")
axs[5,1].set_xlabel(f"RMSE={rmse} PC={round(pc, 2)} MaxError={round(np.max(error))}")

# Show the figure
plt.show()
